<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). Translated by [Eugene Mashkin](https://www.linkedin.com/in/eugene-mashkin-88490883/) (@emashkin). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose. This material is a translated version of the Capstone project (by the same author) from specialization "Machine learning and data analysis" by Yandex and MIPT. No solutions shared.

# <center> Project "Alice". User Identification Based on Visited Websites


In this project, we are going to tackle a user identification problem using their behavioral patterns on the Internet. It is a complicated and interesting problem that combines data analysis and behavioral psychology.
As an example, Yandex solves a mailbox intruder detection problem. In a nutshell, intruder's patterns differ from that of the mailbox owner:

- intruder might not delete emails right after they are read as the mailbox owner would
- intruder might mark emails or even move the cursor differently
- etc.
 
Therefore, the intruder could be detected and thrown out from the mailbox, forcing the user to authenticate via SMS-code. This pilot project is described in the Habrahabr [article](https://habrahabr.ru/company/yandex/blog/230583/) (in Russian). 
 
Similar things are being developed in Google Analytics and, generally, in academia. You can find more by searching for "Traversal Pattern Mining" or  "Sequential Pattern Mining".

<img src='../../img/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

In this project, we are going to solve a similar problem: identify a user given a series of consequently visited. The main idea is the following: Internet users have different patterns in visiting websites: some person might check mailbox first, then read latest football news, and only after that get down to business, another person might get to work right away. 

Our algorithm needs to analyze the sequence of websites consequently visited by a particular person and predict whether this person is Alice or someone else. We will measure ROC AUC. Stay tuned until the end of the course to find out who Alice is.

We will use the data from the paper ["A Tool for Classification of Sequential Data"](http://ceur-ws.org/Vol-1703/paper12.pdf). Even though we can't recommend this paper (the described methods are far from state-of-the-art, it's better to refer to the ["Frequent Pattern Mining"](http://www.charuaggarwal.net/freqbook.pdf) book and the latest ICDM papers), but the dataset is carefully collected and is therefore of interest.

This is data from Blaise Pascal University's proxy-servers. The structure is crystal clear. There is a file `user[USER_ID].csv` for each user (where [USER_ID] - is an id of a particular user). All website visits have the following format: <br><br>

<center>timestamp, visited website</center>

You can download the data using the link provided in the article. Data description could be found there as well. Using the data of all 3000 users is not necessary for this project. Data of 10 and 150 users will be enough. Capstone_user_identification archive [link](https://yadi.sk/d/3gscKIdN3BCASG) (~7 Mb, unziped data ~60 Mb).

In the final project, you'll face an issue that not all operations can be executed within a reasonable time (e.g. it's unlikely you will be able to perform cross-validation grid search over 100 combinations of random forest parameters on this data). Therefore we are going to use two sets of data:
- data of 10 users - we will use it to test and debug our code
- data of 150 users - it will be our main working data

The data has the following
- there are 10 `user[USER_ID].csv` files in `10users` directory
- the same for the `150users` directory - but there 150 files in total
- there is a toy example of 3 users data in the `3users` directory - use it to debug your preprocessing code, which you are supposed to develop further


Your work on this project can be nicely coupled with participating in [this](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle Inclass competition. Check out [mlcourse.ai roadmap](https://mlcourse.ai/roadmap) for more info about grading. 



## <center> Week 1. Preparing the training set

The fisrt part of the project is devoted to data preparation for further descriptive analysis and predictive model development. You need to write a code for data preprocessing and producing a single training set (initially the data is stored in separate files). Also we will learn sparse data format (`Scipy.sparse` matricies), which is well-suited for this purpose.


**Week 1 roadmap**
 - Part 1. Training set preparation
 - Part 2. Working with sparse data format

**Your task**
1. Fill in th missing code in the provided notebook
2. Choose the answers in the [form](https://docs.google.com/forms/d/10yakW9zN85pTVnTo9uStzdAUlqj8nARwIhZxlT4stZk)

**You might find the following materials useful:**
- [Loops](https://www.datacamp.com/community/tutorials/loops-python-tutorial), [functions](https://www.datacamp.com/community/tutorials/functions-python-tutorial), [generators](https://www.learnpython.org/en/Generators), [list_comprehension](https://www.datacamp.com/community/tutorials/python-list-comprehension)
- [Reading and writing data from/to files in python](https://www.datacamp.com/community/tutorials/reading-writing-files-python)
- [Pandas Tutorial: DataFrames in Python](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python)
   
**Besides, we are going to use [`pickle`](https://docs.python.org/2/library/pickle.html), [`glob`](https://docs.python.org/3/library/glob.html) and `Scipy.sparse` ([`csr_matrix`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html) class) python libraries.**

Finally, let's list versions of all main libraries used in this project for better results reproducibility: NumPy, SciPy, Pandas, Matplotlib, Statsmodels and Scikit-learn. To do this use [watermark](https://github.com/rasbt/watermark) extension. It is recommended to use Open Data Science docker container with all  libraries and dependecies preinstalled (details are [here](https://github.com/Yorko/mlcourse.ai/wiki/Software-requirements-and-Docker-container)). 

In [1]:
# pip install watermark
%load_ext watermark

ModuleNotFoundError: No module named 'watermark'

In [2]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

UsageError: Line magic function `%watermark` not found.


In [10]:
# Disable Anaconda warnings
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
# pip install tqdm
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

import glob
from tqdm import tqdm

**Let's look at one of the files containing the data about websites visited by a particular user (user 31).**

In [2]:
# Change the path to data
PATH_TO_DATA = '../../data/capstone_user_identification'

In [3]:
user31_data = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                       '10users/user0031.csv'))

In [4]:
user31_data.head()

,timestamp,site
0,2013-11-15 08:12:07,fpdownload2.macromedia.com
1,2013-11-15 08:12:17,laposte.net
2,2013-11-15 08:12:17,www.laposte.net
3,2013-11-15 08:12:17,www.google.com
4,2013-11-15 08:12:18,www.laposte.net


**Let's define the problem: identify a user given a session containing 10 consequentially visited websites. Each sample is a session of 10 websites consequentially visited by a particular user, features are indexes of these 10 websites (later, we will get a bag of websites applying bag-of-words approach). Target class is the user id.**

### <center>Toy example</center>
**Suppose there are only 2 users, and session length is 2 (websites).**

<center>user0001.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user0002.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Iterate through the first file and assign consequtive site_id-s (numbers) to each new incoming websites: vk.com – gets site_id=1, google.com – gets site_id=2, etc. Then, iterate through the second file. So we collect a bag of websites. 

You are supposed to get the following mapping:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Then your training set should be (target class - user_id):

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>


Here, the first sample is a session containing two websites visited by the first user (target=1). There are `vk.com` and `google.com` in it (site1=1 and site2=2). Four sessions in total. In this example sessions are not overlapping each other, i.e. each website belongs to only one session.

## Part 1. Training set preparation

Implement the function `prepare_train_set`, which takes as an input:
- *path_to_csv_files* - path to the directory containing csv-files
- *session_length* - number of websites in a session

and returns two objects:
- DataFrame with rows corresponding to a unique user session (each of *session_length* websites) and *session_length* columns corresponding to the websites in the session. And there should be one more column - *user_id* which is target class of a sample. Hence the DataFrame should have *session_length + 1* columns.
- Dictionary of websites' frequncies which has {'site_string': [site_id, site_freq]} structure, where `site_string` - original website name, `site_id` - a number the website got be encoded, `site_freq` - total number of website occurrences in all files among all users. For our recent toy example it is:
{'vk.com': (1, 2),<br> 'google.com': (2, 2),<br> 'yandex.ru': (3, 3),<br> 'facebook.com': (4, 1)}

Details:
- You can find an example of function's output below
- Use `glob` (or somthing similar) to iterate through files in directory. For certainty, sort the list of files lexicographically. It is handy to use `tqdm_notebook` (or just `tqdm` in case of a python script) to track the number of loop iterations done
- Create the dictionary of websites' frequencies (with the following structure: {'site_string': (site_id, site_freq)}) and fill it iterating through the file. Start with 1
- It is recommended to give lesser indeces to more frequent websites 
- Don't do entity recognition, consider *google.com*, *http://www.google.com* and *www.google.com* as different websites (you can try entity recognition in scope of your individual work on the project)
- It's likely that the number of records in a file is not divisible by *session_length*. In this case, the last session shall be shorter. Fill empty values with zeros. I.e. if there 24 recods in a file and session_length is 10, then third session will contain 4 websites and its corresponding vector will be:<br> 
[*site1_id*, *site2_id*, *site3_id*, *site4_id*, 0, 0, 0, 0, 0, 0, *user_id*] 
- Some sessions might be repeatitions of previous ones - leave it as is, don't drop duplicates. If there are identical web_sites in two sessions, but these sessions belong to different users, leave it as is as well. It's natural data uncertainty
- Don't keep the website with `site_id`=0 in the dictionary (after your function returns that dictionary).
- It took me 1.7 sec to process 150 files from *capstone_websites_data/150users/*, but, surely, it depends on your implementation of the function and hardware you use. Frankly, it's more likely that your first implementation won't be the most efficient one. As a next step, you can profile your code (especially if you plan to run your code on 3000 users data). Also, an efficient implementation of that function will help you next week.

In [97]:
def prepare_train_set(path_to_csv_files, session_length=10):
    """
    Parameters
    ----------
    path_to_csv_files : str
        path to the directory containing csv-files
    session_length : str
        number of websites in a session

    Returns
    -------
    two objects
        1. DataFrame with rows corresponding to a unique user session
        (each of session_length websites) and session_length columns
        corresponding to the websites in the session. And there
        should be one more column - user_id which is target class
        of a sample. Hence the DataFrame should have session_length + 1
        columns.
        2. Dictionary of websites' frequncies which has
        {'site_string': [site_id, site_freq]} structure,
        where site_string - original website name,
        site_id - a number the website got be encoded,
        site_freq - total number of website occurrences in all files
        among all users. For example:
            {'vk.com': (1, 2),
            'google.com': (2, 2),
            'yandex.ru': (3, 3),
            'facebook.com': (4, 1)}
    """

    # sites columns
    columns = ['site%s' % i for i in range(1, session_length+1)]
    # target: user_id
    columns.append('user_id')
    sites_columns = ['site%s' % i for i in range(1, session_length+1)]

    # result DataFrame
    df = pd.DataFrame(columns=columns)
    # result Dictionary
    sites = {}

    for next_file in tqdm(glob.glob(os.path.join(path_to_csv_files, '*.csv'))):
        #print(next_file)
        # get user id: userNNNN.csv
        user_id = int(next_file[-8:-4])
        #print('user_id=%s' % user_id)
        orig_file_df = pd.read_csv(next_file)

        full_sessions_count = orig_file_df.shape[0] // session_length

        #file_df = pd.DataFrame()

        if full_sessions_count > 0:
            full_sess_df = orig_file_df[:(session_length*full_sessions_count)]['site'].values.flatten().\
                reshape(full_sessions_count, session_length)
            file_df = pd.DataFrame(full_sess_df, columns=sites_columns)
            file_df['user_id'] = user_id

        last_session_part = orig_file_df.shape[0] % session_length

        if last_session_part > 0:
            parted = pd.DataFrame(columns=columns)
            parted = pd.DataFrame(data = [ [''] * (session_length+1) ], columns=columns)
            parted[sites_columns] = ''
            parted['user_id'] = user_id
            parted.iloc[:, :last_session_part] = orig_file_df[-last_session_part:]['site'].values.flatten()
            if full_sessions_count > 0:
                file_df = pd.concat([file_df, parted], ignore_index=True)
            else:
                file_df = parted

        df = pd.concat([df, file_df], ignore_index=True)

#    stat = pd.DataFrame(data=df[sites_columns].values.flatten(), columns=['site'])
#    def sites_ttl(site1):
#        site = site1  #[:1][0]
#        if not sites.get(site):
#            if site == '':
#                sites[''] = (0, 0)
#            else:
#                cur_id = sites_ttl.cur_id
#                sites_ttl.cur_id = sites_ttl.cur_id + 1
#                sites[site] = (cur_id, 0)
#                ++cur_id
#
#        sites[site] = (sites[site][0], sites[site][1] + 1)
#        return sites[site][0]
#
#    sites_ttl.cur_id = 1
#
#    stat['site'].apply(sites_ttl)


    stat = pd.DataFrame(data=df[sites_columns].values.flatten(), columns=['site'])
    values = stat['site'].value_counts(sort=True)
    stat = pd.DataFrame(data=list(zip(values.index.values, values.values)), columns=['site', 'count'])
    empte_pos=stat.loc[stat['site']=='']

    if not empte_pos.empty:
        empte_pos=empte_pos.index[0]
        empty_value = stat.iloc[empte_pos].copy()
        stat.drop(empte_pos, inplace=True)
        # need 'empty'?
        stat = stat.append(empty_value)

    sites = { site: (site_id+1, site_freq) for site, site_freq, site_id in zip(stat['site'].values, stat['count'].values, range(stat.shape[0])) }

    # empty?
    if sites.get(''):
        sites[''] = (0, sites[''][1])

    df[sites_columns] = df[sites_columns].apply(lambda sess: [sites.get(key)[0] for key in sess.values])

    del sites['']

    return df, sites

**Apply your function to the toy example and make sure that everything works fine:**

In [98]:
!cat $PATH_TO_DATA/3users/user0001.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,geo.mozilla.org
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,accounts.google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,apis.google.com
2013-11-15 11:41:35,plus.google.com
2013-11-15 12:40:35,vk.com
2013-11-15 12:40:37,google.com
2013-11-15 12:40:40,google.com
2013-11-15 12:41:35,google.com


In [99]:
!cat $PATH_TO_DATA/3users/user0002.csv

timestamp,site
2013-11-15 09:28:17,vk.com
2013-11-15 09:33:04,oracle.com
2013-11-15 09:52:48,football.kulichki.ru
2013-11-15 11:37:26,football.kulichki.ru
2013-11-15 11:40:32,oracle.com


In [100]:
!cat $PATH_TO_DATA/3users/user0003.csv

timestamp,site
2013-11-15 09:28:17,meduza.io
2013-11-15 09:33:04,google.com
2013-11-15 09:52:48,oracle.com
2013-11-15 11:37:26,google.com
2013-11-15 11:40:32,oracle.com
2013-11-15 11:40:34,google.com
2013-11-15 11:40:35,google.com
2013-11-15 11:40:37,mail.google.com
2013-11-15 11:40:40,yandex.ru
2013-11-15 11:41:35,meduza.io
2013-11-15 12:28:17,meduza.io
2013-11-15 12:33:04,google.com
2013-11-15 12:52:48,oracle.com


In [104]:
%%time
train_data_toy, site_freq_3users = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 
                                                     session_length=10)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 73.17it/s]


Wall time: 63 ms


In [105]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,user_id
0,4,2,2,10,2,1,9,6,8,7,1
1,4,1,1,1,0,0,0,0,0,0,1
2,4,2,5,5,2,0,0,0,0,0,2
3,3,1,2,1,2,1,1,6,11,3,3
4,3,1,2,0,0,0,0,0,0,0,3


Websites' frequencies (second element of a tuple) should be exactly the same. Numeration (websites order) (first elements of tuples) may vary.

In [106]:
site_freq_3users

{'google.com': (1, 9),
 'oracle.com': (2, 8),
 'meduza.io': (3, 3),
 'vk.com': (4, 3),
 'football.kulichki.ru': (5, 2),
 'mail.google.com': (6, 2),
 'plus.google.com': (7, 1),
 'apis.google.com': (8, 1),
 'accounts.google.com': (9, 1),
 'geo.mozilla.org': (10, 1),
 'yandex.ru': (11, 1)}

Apply your function to the data with 10 users.

**<font color='red'> Question 1. </font>How many unique sessions with length of 10 websites are there in 10users data?**

In [116]:
%%time
train_data_10users, site_freq_10users = prepare_train_set(os.path.join(PATH_TO_DATA, '10users'), 
                                                     session_length=10)
#print(train_data_10users.head())
#train_data_10users[:-1].head()
print(train_data_10users.iloc[:, :-1].drop_duplicates().count()[0])  # 13183

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 43.67it/s]


13183
Wall time: 368 ms


**<font color='red'> Questioin 2. </font>How many unique websites are there in 10users data?**

In [117]:
len(site_freq_10users) # 4913

4913

Apply your function to the data with 150 users.

**<font color='red'> Question 3. </font> How many unique sessions with length of 10 websites are there in 150users data?**

In [118]:
%%time
train_data_150users, site_freq_150users = prepare_train_set(os.path.join(PATH_TO_DATA, '150users'), 
                                                     session_length=10)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:06<00:00, 13.92it/s]


Wall time: 8.01 s


**<font color='red'> Question 4. </font> How many unique websites are there in 150users data?**

In [119]:
print(len(site_freq_150users)) # 27797

27797

**<font color='red'> Question 5. </font> 
Which of these websites is NOT in top-10 most visited websites among 150 users?**
- www.google.fr
- www.youtube.com
- safebrowsing-cache.google.com
- www.linkedin.com

In [122]:
print(site_freq_150users)

{'www.google.fr': (1, 64785), 'www.google.com': (2, 51320), 'www.facebook.com': (3, 39002), 'apis.google.com': (4, 29983), 's.youtube.com': (5, 29102), 'clients1.google.com': (6, 25087), 'mail.google.com': (7, 19072), 'plus.google.com': (8, 18467), 'safebrowsing-cache.google.com': (9, 17960), 'www.youtube.com': (10, 16319), 'twitter.com': (11, 16219), 'platform.twitter.com': (12, 15317), 's-static.ak.facebook.com': (13, 15048), 'accounts.google.com': (14, 13855), 'www.bing.com': (15, 13797), 'static.ak.facebook.com': (16, 13117), 'i1.ytimg.com': (17, 13117), 'download.jboss.org': (18, 11740), 'api.twitter.com': (19, 9350), 'safebrowsing.clients.google.com': (20, 8981), 'r1---sn-gxo5uxg-jqbe.googlevideo.com': (21, 8579), 'fr.openclassrooms.com': (22, 8100), 'ajax.googleapis.com': (23, 7811), 'r3---sn-gxo5uxg-jqbe.googlevideo.com': (24, 7482), 'drive.google.com': (25, 7341), 'r2---sn-gxo5uxg-jqbe.googlevideo.com': (26, 7053), 'r4---sn-gxo5uxg-jqbe.googlevideo.com': (27, 7039), 's.ytimg.c

**Write dataframes to csv files for further analysis.**

In [ ]:
train_data_10users.to_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), 
                        index_label='session_id', float_format='%d')
train_data_150users.to_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), 
                         index_label='session_id', float_format='%d')

## Part 2. Working with sparse data format

If you think carefully, the features we've got - *site1*, ..., *site10* - won't work at all in a classification problem. But if we use the Bag of Words idea from text analysis - it's another story. Create new matrices with sessions as rows, and site_id as columns. The intersection of row $i$ and column $j$ is $n_{ij}$ - the number of website $j$ occurrences in session $i$. We are going to do this using sparse Scipy matrices - [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Read the docs, figure out how to use sparse matrices and create such matrices for our data. First, debug your code on toy example and then apply it to 10users and 150users data.

Note, that in short sessions (less than 10 websites) there are trailing zeros, hence the meaning of the first feature (number of 0 occurrences in a session) differs from others (number of $i$-th website occurrences in a session). Therefore you need to drop the first column in a Dataframe.

In [ ]:
X_toy, y_toy = train_data_toy.iloc[:, :-1].values, train_data_toy.iloc[:, -1].values

In [ ]:
X_toy

In [ ]:
X_sparse_toy = csr_matrix ''' YOUR CODE IS HERE '''   

**Number of columns in `X_sparse_toy` should be 11, because in the toy example 3 users visited 11 unique websites.**

In [ ]:
X_sparse_toy.todense()

In [ ]:
X_10users, y_10users = train_data_10users.iloc[:, :-1].values, \
                       train_data_10users.iloc[:, -1].values
X_150users, y_150users = train_data_150users.iloc[:, :-1].values, \
                         train_data_150users.iloc[:, -1].values

In [ ]:
X_sparse_10users = ''' YOUR CODE IS HERE '''
X_sparse_150users = ''' YOUR CODE IS HERE '''

**Save these sparse matrices with [pickle](https://docs.python.org/2/library/pickle.html) (serialization in Python), and *y_10users, y_150users* - target variables (user_id-s) for our 10users and 150users data. The fact that the names of these matrices start with X and y implies that we are going to check our first classification models on them.
Finally, save frequency dictionaries for 3users, 10users and 150users data.**

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_3users.pkl'), 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_150users.pkl'), 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl, protocol=2)

**Just in case doublecheck that number of columns in sparse matrices `X_sparse_10users` and `X_sparse_150users` equals to the number of unique websites in 10users and 150users data evaluated earlier.**

In [ ]:
assert X_sparse_10users.shape[1] == len(site_freq_10users)

In [ ]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)

Next week we will preprocess the data a bit and check first hypotheses regarding our observations.